In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.qc.scanpy_qc import scanpy_qc
from tools.qc.dropkick_qc import dropkick_qc
from tools.formating.formating import *
# from tools.run_qc import run_qc

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.2.2 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.8
scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.2.2 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.8


In [2]:
dataset='RDS'
# input_path='/ps/ai-ready/data/filtered_gene_bc_matrices/hg19/'
# methods=None
# input_path='/ps/ai-ready/data/kbcfh/loomDataset/867236c9-e8c8-46a4-971a-3c046b1d4328.loom'
# input_path='/ps/ai-ready/data/kbcfh/gzDataset/counts.csv.gz'
input_path='/ps/ai-ready/data/kbcfh/TungDataset/tung.rds'
methods='Scanpy'
path_of_scrublet_calls=os.path.abspath('../tools/qc/scrublet_calls.tsv')
print(path_of_scrublet_calls)

/ps/Machine-learning-development-environment-for-single-cell-sequencing-data-analyses/api/tools/qc/scrublet_calls.tsv


In [ ]:
# run_qc(dataset, input, output, methods, path_of_scrublet_calls='./scrublet_calls.tsv', show_error=True)

In [ ]:
os.path.isdir(input_path)

In [ ]:
os.path.dirname(input_path)

In [ ]:
adata_path = os.path.splitext(input_path)[0] + '.h5ad'
adata_path

In [ ]:
# adata = sc.read_loom(input_path)
# adata

In [ ]:
adata = sc.read_text(input_path)
adata

In [ ]:
suffix = input_path.endswith('.gz')
suffix

In [ ]:
import pyreadr

result = pyreadr.read_r(input_path)
result

## Scanpy

In [3]:
adata = load_anndata(input_path)
adata

/ps/ai-ready/data/kbcfh/TungDataset/tung.rds

R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> rmarkdown::render('/ps/Machine-learning-development-environment-for-single-cell-sequencing-data-analyses/api/tools/formating/convert_to_anndata.Rmd', params=list(path='/ps/ai-ready/data/kbcfh/TungDataset/tung.rds'), output_file='/ps/ai-ready/data/kbcfh/TungDat



processing file: convert_to_anndata.Rmd


                                                                                                      
/usr/bin/pandoc +RTS -K512m -RTS convert_to_anndata.knit.md --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output /ps/ai-ready/data/kbcfh/TungDataset/file_conversion_report.html --lua-filter /usr/local/lib/R/site-library/rmarkdown/rmarkdown/lua/pagebreak.lua --lua-filter /usr/local/lib/R/site-library/rmarkdown/rmarkdown/lua/latex-div.lua --embed-resources --standalone --variable bs3=TRUE --section-divs --template /usr/local/lib/R/site-library/rmarkdown/rmd/h/default.html --no-highlight --variable highlightjs=1 --variable theme=bootstrap --mathjax --variable 'mathjax-url=https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML' --include-in-header /tmp/RtmpwDtxjB/rmarkdown-str68864b738d8.html 
> 
> 


output file: convert_to_anndata.knit.md


Output created: /ps/ai-ready/data/kbcfh/TungDataset/file_conversion_report.html


0


AnnData object with n_obs × n_vars = 670 × 13873
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'individual', 'replicate', 'well', 'batch', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'altexps_ERCC_sum', 'altexps_ERCC_detected', 'altexps_ERCC_percent', 'total', 'discard'
    var: 'SYMBOL', 'mean', 'detected', 'discard'
    obsm: 'X_PCA', 'X_TSNE'

In [4]:
type(adata)

anndata._core.anndata.AnnData

In [ ]:
adata is None

In [7]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr

In [ ]:
output = get_output_path(dataset, input_path, method='scanpy', format = "AnnData")

In [ ]:
adata.X.shape

In [5]:
type(adata.X)

scipy.sparse._csr.csr_matrix

In [14]:
adata.X.stats

AttributeError: stats not found

In [9]:
pd.DataFrame.sparse.from_spmatrix(adata.X).max()

0         2.0
1        11.0
2         4.0
3        13.0
4        17.0
         ... 
13868     8.0
13869     9.0
13870    18.0
13871     8.0
13872    39.0
Length: 13873, dtype: float64

In [18]:
adata.X.shape

(670, 13873)

In [19]:
n_samples = _num_samples(adata.X)
n_samples

NameError: name '_num_samples' is not defined

In [15]:
adata = scanpy_qc(adata)
 # Save AnnData object
adata.write_h5ad(output, compression='gzip')
print("AnnData object for Scanpy QC is saved successfully")

normalizing counts per cell
    finished (0:00:00)


ValueError: Found array with 0 sample(s) (shape=(0, 13873)) while a minimum of 1 is required.

In [ ]:
adata.var_names_make_unique()
adata

In [ ]:
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                          min_gene_variability_pctl=85, n_prin_comps=30)

In [ ]:
scrub.plot_histogram()

In [ ]:
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
scrub.plot_embedding('UMAP', order_points=True);

In [ ]:
adata.obs

In [ ]:
adata.obs['predicted_doublets'].value_counts()

In [ ]:
scrublet_path = os.path.join(os.path.dirname(input_path), "scrublet_calls.tsv")
scrublet_path

In [ ]:
df_predicted_doublets = adata.obs.iloc[:, -2:]
df_predicted_doublets 

In [ ]:
df_

In [ ]:
pd.DataFrame(adata.obs.iloc[:, -2:]).to_csv(scrublet_path,sep = '\t',header = False)

## Dropkick

In [ ]:
adata = load_anndata(input_path)
adata

In [ ]:
adata = dropkick_qc(adata)
output_path = get_output_path(dataset, input_path, method='dropkick')
# Save AnnData object
adata.write_h5ad(output_path, compression='gzip')
print("AnnData object for Dropkick QC is saved successfully")

## BIOCONDUCTOR

In [ ]:
output_path = get_output_path(dataset, input_path, method='Bioconductor', format='SingleCellExperiment')

In [ ]:
report_path = get_report_path(dataset, output_path, "Bioconductor")
print(report_path)

In [ ]:
bioconductor_path = os.path.abspath("../tools/qc/bioconductor_qc.Rmd")
print(bioconductor_path)

In [ ]:
s = subprocess.call(["R -e \"rmarkdown::render('" + bioconductor_path + "', params=list(dataset='" + str(dataset) + "', input_path='" + input_path + "', output='" + output_path + "', idtype='SYMBOL', colour_by=NULL, shape_by_1='NULL', shape_by_2='NULL',output_format='SingleCellExperiment'), output_file='" + report_path + "')\""], shell = True)
print(s)

## SEURAT

In [ ]:
output_path = get_output_path(dataset, input_path, method='Seurat', format='Seurat')
report_path = get_report_path(dataset, output_path, "Seurat")
print(report_path)

In [ ]:
seurat_path = os.path.abspath("../tools/qc/seurat_qc.Rmd")
s = subprocess.call(["R -e \"rmarkdown::render('" + seurat_path + "', params=list(dataset='" + str(dataset) + "', input='" + input_path + "', output='" + output_path + "', output_format='Seurat', path_of_scrublet_calls='" + path_of_scrublet_calls + "'), output_file='" + report_path + "')\""], shell = True)
print(s)